<h2>Capstone Project<h2>

This notebook is for Capstone project and we will be using <code>Pandas</code> in the Python Programming Language and Various machine learning Techniques to deliver the end outcome.

<h3>Battele of Neighborhood: To Find the Best neighborhood suitable for Resturant Business!<h3>

<h2> 3. Explore and cluster the neighborhoods in Toronto <\h2>

<h3> Download/Import the dependencies Liabraries <\h3>

In [1]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [170]:
# Read the Final Dataset Saved as Final_Dataset_with_Geo_coordinates.csv

import pandas as pd
import numpy as np

df_FinalDataset = pd.read_csv('Final_Dataset_with_Geo_coordinates.csv')
df_FinalDataset.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h3> Use geopy library to get the latitude and longitude values of Toronto City.<\h3>

In [171]:
address = 'Toronto'

#geolocator = Nominatim(user_agent="ny_explorer")
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are:', latitude, longitude)

The geograpical coordinate of Toronto are: 43.6534817 -79.3839347


<h3> Create a map of Toronto with neighborhoods superimposed on top.<\h3>

In [172]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_FinalDataset['Latitude'], df_FinalDataset['Longitude'], df_FinalDataset['Borough'], df_FinalDataset['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    #print(label)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3> Simplify the above map and segment and cluster only the neighborhoods in Downtown Toronto <\h3>

In [195]:
df_Toronoto_Downtown = df_FinalDataset[df_FinalDataset['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
df_Toronoto_Downtown.shape

(19, 5)

Get the Geo Coordinates of Toronto Downtown

In [174]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The Geo coordinate of Toronto Downtown are:', latitude, longitude)

The Geo coordinate of Toronto Downtown are: 43.6541737 -79.38081164513409


In [175]:
# create map of Toronto Downtown using latitude and longitude values
map_torontodowntown = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronoto_Downtown['Latitude'], df_Toronoto_Downtown['Longitude'], df_Toronoto_Downtown['Borough'], df_Toronoto_Downtown['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torontodowntown)  
    
map_torontodowntown

<h3> Define Foursqaure credentials <\h3>

In [176]:
CLIENT_ID = 'DHN0LHBZZCEPHH31FNDPKBC4AFSKYVC3NVZM2AJRB4MVE0C5' # your Foursquare ID
CLIENT_SECRET = 'PQHOCSFLIZVOBZ2IWZWBVCAZV2NI2X2U1GZPR32BRKYJSUTD' # your Foursquare Secret
VERSION = '20200521'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DHN0LHBZZCEPHH31FNDPKBC4AFSKYVC3NVZM2AJRB4MVE0C5
CLIENT_SECRET:PQHOCSFLIZVOBZ2IWZWBVCAZV2NI2X2U1GZPR32BRKYJSUTD


<h3> Explore the first Neighborhood in our Dataset <\h3>

In [177]:
Neighborhood = df_Toronoto_Downtown.loc[0,'Neighborhood']

neighborhood_latitude = df_Toronoto_Downtown.loc[0,'Latitude']
neighborhood_longitude = df_Toronoto_Downtown.loc[0,'Longitude']

print('{} Neighborhood Longitude is {} and lattitude is{} .'.format(Neighborhood,neighborhood_longitude,neighborhood_latitude))

Regent Park, Harbourfront Neighborhood Longitude is -79.3606359 and lattitude is43.6542599 .


<h3> Get the top 100 venues that are in Regent Park within a radius of 500 meters. <\h3>

In [178]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# display URL
url

'https://api.foursquare.com/v2/venues/explore?&client_id=DHN0LHBZZCEPHH31FNDPKBC4AFSKYVC3NVZM2AJRB4MVE0C5&client_secret=PQHOCSFLIZVOBZ2IWZWBVCAZV2NI2X2U1GZPR32BRKYJSUTD&v=20200521&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [179]:
# Examine the Result set
results = requests.get(url).json()
#results

In [180]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [181]:
#ven = results['response']['groups'][0]['items']
#nearby_venues = json_normalize(ven)
#nearby_venues.loc[1:2,'venue.name']

In [197]:
# clean the json and structure it into a pandas dataframe.

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)
nearby_venues.shape

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


(49, 4)

In [183]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

49 venues were returned by Foursquare.


<h3> Explore Neighborhoods in Toronto Downtown <\h3>

<h3> Create a function to repeat the same process to all the neighborhoods in Toronoto Downtown. <\h3>

In [184]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h3> Run the above function on each neighborhood and create a new dataframe called Tornoto_Downtown_Venues.<\h3>

In [185]:
Tornoto_Downtown_Venues = getNearbyVenues(names=df_Toronoto_Downtown['Neighborhood'],
                                   latitudes=df_Toronoto_Downtown['Latitude'],
                                   longitudes=df_Toronoto_Downtown['Longitude']
                                  )



#Tornoto_Downtown_Venues['Venue Category'].unique()

In [186]:
# Count of Distinct Neighborhood in Toronto Downtown 

len(Tornoto_Downtown_Venues['Neighborhood'].unique())

19

In [187]:
df_RstaurantOnly = Tornoto_Downtown_Venues[Tornoto_Downtown_Venues['Venue Category'].str.contains('Restaurant')]
df_AllOthers = Tornoto_Downtown_Venues[Tornoto_Downtown_Venues['Venue Category'].str.contains('Restaurant') == False]

df_RstaurantOnly.head()
#df_AllOthers.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
21,"Regent Park, Harbourfront",43.654260,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
35,"Regent Park, Harbourfront",43.654260,-79.360636,El Catrin,43.650601,-79.358920,Mexican Restaurant
42,"Regent Park, Harbourfront",43.654260,-79.360636,Flame Shack,43.656844,-79.358917,Restaurant
50,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Mercatto,43.660391,-79.387664,Italian Restaurant


In [188]:
# Unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(df_RstaurantOnly['Venue Category'].unique())))
print('There are {} uniques categories.'.format(len(df_AllOthers['Venue Category'].unique())))



There are 41 uniques categories.
There are 169 uniques categories.


<h3> Analyse Each Neighborhood for Restaurant and Other Facilities<\h3>

In [189]:
# one hot encoding for both Restaurants only and All Others venue Category Dataset

RestaurantOnly_onehot = pd.get_dummies(df_RstaurantOnly[['Venue Category']], prefix="", prefix_sep="")
AllOthers_onehot = pd.get_dummies(df_AllOthers[['Venue Category']], prefix="", prefix_sep="")

RestaurantOnly_onehot = pd.concat([df_RstaurantOnly['Neighborhood'], RestaurantOnly_onehot], axis = 1)
AllOthers_onehot = pd.concat([df_AllOthers['Neighborhood'], AllOthers_onehot.drop(['Neighborhood'], axis = 1)], axis = 1)

#RestaurantOnly_onehot.head()
RestaurantOnly_onehot[['Neighborhood']]

,Neighborhood
5,"Regent Park, Harbourfront"
21,"Regent Park, Harbourfront"
35,"Regent Park, Harbourfront"
42,"Regent Park, Harbourfront"
50,"Queen's Park, Ontario Provincial Government"
...,...
1197,Church and Wellesley
1200,Church and Wellesley
1204,Church and Wellesley
1212,Church and Wellesley


In [190]:
RestaurantOnly_onehot.shape

RestaurantOnly_onehot.head()
AllOthers_onehot.head()


,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Tea Room,Theater,Trail,Train Station,Video Game Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3>Let's Identify the Neighborhoods and it's Restaurant Business Around<\h3>


In [191]:
# Group the Neighborhood one hot dataset,  to get the Number for Restaurant in Neighborhod 
Neigh_Restaurant_groupby = RestaurantOnly_onehot.groupby(['Neighborhood']).sum().reset_index()


# Get the Number of Restaurants in the Neighborhood
Neigh_Restaurant_groupby['RestaurantCnt'] = Neigh_Restaurant_groupby.sum(axis=1)

# Create the Neighnborhood Restaurant Index
Neighborhood_Restaurant= Neigh_Restaurant_groupby[['Neighborhood','RestaurantCnt']].sort_values(by = 'RestaurantCnt').reset_index()

Neighborhood_Restaurant.drop(['index'],axis=1,inplace=True)
Neighborhood_Restaurant.head()


,Neighborhood,RestaurantCnt
0,Christie,2
1,"Regent Park, Harbourfront",4
2,"Queen's Park, Ontario Provincial Government",4
3,"University of Toronto, Harbord",10
4,"St. James Town, Cabbagetown",10


<h3>Find out the Other Venues or Happening Places (Other than Restaurant) around Neighborhoods<\h3>

In [192]:
# Group the Neighborhood one hot dataset,  to get all other venuws (other than Restaurant) in the Neighborhod
# THis is to identify the happening places and rate them accordingly

Neighborhood_AllOthers_groupby = AllOthers_onehot.groupby(['Neighborhood']).sum().reset_index()

# Get the Number of Restaurants in the Neighborhood
Neighborhood_AllOthers_groupby['OthVenueCnt'] = Neighborhood_AllOthers_groupby.sum(axis=1)

# Create the Neighnborhood Restaurant Index
Neighborhood_AllOther= Neighborhood_AllOthers_groupby[['Neighborhood','OthVenueCnt']].sort_values(by = 'OthVenueCnt').reset_index()

Neighborhood_AllOther.drop(['index'],axis=1,inplace=True)
Neighborhood_AllOther.head()

,Neighborhood,OthVenueCnt
0,Rosedale,4
1,"CN Tower, King and Spadina, Railway Lands, Har...",15
2,Christie,15
3,"University of Toronto, Harbord",24
4,"Queen's Park, Ontario Provincial Government",30


<h3> Calculate the Restaurant Index based on the Places around and in business Restaurants in the Neighborhood<\h3>

In [193]:
print('Neighborhood_Restaurant_index size is =',Neighborhood_Restaurant.shape)
print('Neighborhood_AllOther_index size is =',Neighborhood_AllOther.shape)

Final_Analysis_Data = Neighborhood_AllOther.set_index('Neighborhood').join(Neighborhood_Restaurant.set_index('Neighborhood')).reset_index()

# Replace missing value with 0 (Zero)
Final_Analysis_Data['RestaurantCnt'].replace(np.NaN,0,inplace = True)
Final_Analysis_Data['RestaurantCnt'] = Final_Analysis_Data['RestaurantCnt'].astype(int)

# Calculate the Restaurant Weightage Index in comparision with all other Venues Around
Final_Analysis_Data['Restaurant_index'] = 1 - (Final_Analysis_Data['RestaurantCnt']/Final_Analysis_Data['OthVenueCnt'])

Final_Analysis_Data = Final_Analysis_Data.sort_values(by = 'Restaurant_index', ascending = False)
Final_Analysis_Data.reset_index (inplace = True)
Final_Analysis_Data.drop(['index'],axis=1,inplace = True)



Neighborhood_Restaurant_index size is = (17, 2)
Neighborhood_AllOther_index size is = (19, 2)


<h3> Top 5 Neighborhood best suitable to start Restaurant Business <\h3>

In [194]:
# Select the Top 5 Locations suitable for Restaurant Business based on their Restauran Index values
Final_Analysis_Data.head()


,Neighborhood,OthVenueCnt,RestaurantCnt,Restaurant_index
0,Rosedale,4,0,1.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",15,0,1.000000
2,"Regent Park, Harbourfront",45,4,0.911111
3,Christie,15,2,0.866667
4,"Queen's Park, Ontario Provincial Government",30,4,0.866667


<h4> Based on the above Analysis, My recomendation for a New Restaurant Business would be "Regent Park" as there are many great happening places around but only 4 Restaurants in top 100 list.  <\h4>